## Imports

In [1]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [2]:
cwd = os.getcwd()

In [3]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import financialstatements

## Config

In [5]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

In [6]:
JOURNAL_ENTRIES = os.path.join(cwd, config.get("data_outputs_directory", "JOURNAL_ENTRIES"))
MONTH_DATA = config.get('table_files', 'MONTH_DATA')

In [7]:
je_paths = financialstatements.creating_input_paths(JOURNAL_ENTRIES)

In [8]:
df = financialstatements.creating_df(je_paths)

In [9]:
month_df = pd.read_excel(MONTH_DATA)

## ETL

In [10]:
df['Date'] = pd.to_datetime(df['Date'])

In [11]:
df = df.sort_values(by=['Date', 'Transaction_ID']).reset_index(drop=True)

In [253]:
df['Month_Num'] = df['Date'].dt.month

In [254]:
df = pd.merge(df, 
              month_df, 
              on='Month_Num', 
              how='left')

In [255]:
months = df['Month'].unique()

In [256]:
GENERAL_LEDGER = os.path.join(cwd, config.get("data_outputs_directory", "GENERAL_LEDGER"))

In [258]:
month_num = 1
for month in months:
    file_df = df[df['Month'] == month]
    file_df = file_df[['Transaction_ID', 'Date', 'GL_Code','Account', 'Description', 'DEBIT', 'CREDIT']]
    file_name = f'{month_num}_general_ledger_{month}.csv'
    file_df.to_csv(os.path.join(GENERAL_LEDGER, file_name), index=False)
    month_num += 1